In [2]:
import torch
import pandas as pd
import numpy as np
import datasets
from load_models_and_data import load_vocabulary, load_embeddings, text_to_embeddings, calc_cosine_sim, calculate_embeddings, create_packed_batch
from tqdm import tqdm
tqdm.pandas()
#from TwoTowerNN import QryTower, DocTower, TripletEmbeddingDataset, run_hyperparameter_tuning
from TwinTowerGRU import QryTower, DocTower, EmbeddingTripletDataset, run_hyperparameter_tuning
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import Dataset, DataLoader,  SubsetRandomSampler
from sklearn.model_selection import KFold, train_test_split
import os
import wandb
from dotenv import load_dotenv
import torch.nn.functional as F

c:\Users\nnamd\OneDrive\Python_learning\MLX\MLX_Week2\mlx_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


API key loaded successfully


wandb: Currently logged in as: nnamdi-odozi (nnamdi-odozi-ave-actuaries) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
from datasets import load_dataset

# Loading datasets from Hugging Face
ds_soft_neg = load_dataset("cocoritzy/week_2_triplet_dataset_soft_negatives")
ds_hard_neg = load_dataset("cocoritzy/week_2_triplet_dataset_hard_negatives")


In [5]:
# Paths to your files
embeddings_path = "./downloaded_model/glove_embeddings.pt" #set this to either own-trained cbow ones or to glove pre-trained ones
vocab_path = "./downloaded_model/glove_ids_to_words.csv"

# Load embeddings and vocabulary
print("Loading embeddings and vocabulary...")
embeddings = load_embeddings(embeddings_path)
word_to_idx = load_vocabulary(vocab_path)

print(f"Loaded embeddings with shape: {embeddings.shape}")
print(f"Loaded vocabulary with {len(word_to_idx)} tokens")

# Example usage (uncomment when ready to test)
sample_text = "This is a test sentence"
embeddings_result, length = text_to_embeddings(sample_text, word_to_idx, embeddings, is_query=True)
print(f"Embedded text shape: {embeddings_result.shape}")

# Testing - Set numpy print options
np.set_printoptions(precision=4, suppress=True, threshold=10)  # threshold limits number of elements shown
numpy_array = embeddings_result.detach().numpy()
print("Embedding array with custom formatting:")
print(numpy_array)
print("Length is:", length)


Loading embeddings and vocabulary...
Loaded embeddings with shape: torch.Size([400000, 100])
Loaded vocabulary with 399998 tokens
Embedded text shape: torch.Size([26, 100])
Embedding array with custom formatting:
[[ 0.2616  0.4472 -0.0968 ... -0.4503  0.4952 -0.203 ]
 [ 0.1372 -0.5429  0.1942 ... -0.5206  0.2543 -0.2376]
 [-0.3046 -0.2365  0.1758 ... -0.8456 -0.0354  0.1704]
 ...
 [ 0.      0.      0.     ...  0.      0.      0.    ]
 [ 0.      0.      0.     ...  0.      0.      0.    ]
 [ 0.      0.      0.     ...  0.      0.      0.    ]]
Length is: 5


In [6]:
sample_text = ""
embeddings_result, length = text_to_embeddings(sample_text, word_to_idx, embeddings, is_query=True)
print(f"Embedded text shape: {embeddings_result.shape}")

np.set_printoptions(precision=4, suppress=True, threshold=10)  # threshold limits number of elements shown
numpy_array = embeddings_result.detach().numpy()
print("Embedding array with custom formatting:")
print(numpy_array)
print("Length is:", length)


Embedded text shape: torch.Size([26, 100])
Embedding array with custom formatting:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Length is: 0


In [7]:
df_soft_neg  = pd.DataFrame(ds_soft_neg['train'])
df_hard_neg  = pd.DataFrame(ds_hard_neg['train'])

In [8]:
embedded_query, length = text_to_embeddings(df_soft_neg['query'][0], word_to_idx, embeddings, is_query=True)
embedded_positive, length = text_to_embeddings(df_soft_neg['positive_passage'][0], word_to_idx, embeddings, is_query=False)
embedded_negative, length = text_to_embeddings(df_soft_neg['negative_passage'][0], word_to_idx, embeddings, is_query=False)

print(embedded_positive.shape)
print(embedded_negative.shape)

torch.Size([201, 100])
torch.Size([201, 100])


In [9]:
a = embedded_query.mean(dim=0)
b = embedded_positive.mean(dim=0)
c = embedded_negative.mean(dim=0)
a.shape


torch.Size([100])

In [ ]:
import torch.nn.functional as F

cosine_similarity = F.cosine_similarity(a, c, dim=0)
print(f"Cosine similarity between query and positive passage: {cosine_similarity.item()}")

In [ ]:

# # Process the dataframe using apply just for first five rows
# print("Calculating similarities... This may take a while depending on dataframe size.")
# similarities = df_soft_neg[0:5].progress_apply(
#     lambda row: calculate_similarities(row, word_to_idx, embeddings), 
#     axis=1
# )

# # Join the similarities to the dataframe
# df_soft_neg_ext = pd.concat([df_soft_neg[0:5], similarities], axis=1)

# # Show a sample of the results
# #print(df_soft_neg_ext[['query_pos_sim', 'query_neg_sim', 'pos_neg_sim']].head())
#print(df_soft_neg_ext.head())
#print(df_soft_neg_ext.columns)

In [10]:

# Process the dataframe using apply
print("Calculating embeddings... This may take a while depending on dataframe size.")
embeddings_padded = df_soft_neg.progress_apply(
    lambda row: calculate_embeddings(row, word_to_idx, embeddings), 
    axis=1
)

# Join the similarities to the dataframe
df_soft_neg_ext = pd.concat([df_soft_neg, embeddings_padded], axis=1)
print(df_soft_neg_ext.head())
# Show a sample of the results
#print(df_soft_neg_ext[['query_pos_sim', 'query_neg_sim', 'pos_neg_sim']].head())

#print(df_soft_neg_ext[['query_pos_sim', 'query_neg_sim', 'pos_neg_sim']].mean())

# Calculate how often the positive passage is ranked higher than negative
#higher_count = (df_soft_neg_ext['query_pos_sim'] > df_soft_neg_ext['query_neg_sim']).sum()
#total = len(df_soft_neg_ext)
#print(f"\nPositive passage ranked higher than negative: {higher_count} out of {total} ({higher_count/total:.2%})")



Calculating embeddings... This may take a while depending on dataframe size.


100%|██████████| 79704/79704 [04:56<00:00, 269.14it/s]


   query_id                                              query  \
0     19699                                        what is rba   
1     19700                       was ronald reagan a democrat   
2     19701  how long do you need for sydney and surroundin...   
3     19702                    price to install tile in shower   
4     19703                    why conversion observed in body   

                                    positive_passage  \
0  Results-Based Accountability® (also known as R...   
1  From Wikipedia, the free encyclopedia. A Reaga...   
2  Sydney is the capital city of the Australian s...   
3  1 Install ceramic tile floor to match shower-A...   
4  Conversion disorder is a type of somatoform di...   

                                    negative_passage  negative_from_query_id  \
0  I finally found some real salary data for phys...                   86595   
1  The Pacific Ocean lies to the east while the S...                   66360   
2  Probiotics are found in

In [11]:
df_soft_neg_ext[0:1]

,query_id,query,positive_passage,negative_passage,negative_from_query_id,query_emb,query_length,pos_emb,pos_length,neg_emb,neg_length
0,19699,what is rba,Results-Based Accountability® (also known as R...,I finally found some real salary data for phys...,86595,"[[tensor(0.0424), tensor(-0.5220), tensor(0.40...",3,"[[tensor(0.0448), tensor(0.0507), tensor(0.194...",110,"[[tensor(0.0302), tensor(0.4461), tensor(0.431...",67


In [ ]:
# Process the dataframe using apply
print("Calculating embeddings... This may take a while depending on dataframe size.")
embeddings_padded = df_hard_neg.progress_apply(
    lambda row: calculate_embeddings(row, word_to_idx, embeddings), 
    axis=1
)

# Join the similarities to the dataframe
df_hard_neg_ext = pd.concat([df_hard_neg, embeddings_padded], axis=1)
print(df_hard_neg_ext.head())
# Show a sample of the results
#print(df_hard_neg_ext[['query_pos_sim', 'query_neg_sim', 'pos_neg_sim']].head())

#print(df_hard_neg_ext[['query_pos_sim', 'query_neg_sim', 'pos_neg_sim']].mean())

# Calculate how often the positive passage is ranked higher than negative
#higher_count = (df_hard_neg_ext['query_pos_sim'] > df_hard_neg_ext['query_neg_sim']).sum()
#total = len(df_hard_neg_ext)
#print(f"\nPositive passage ranked higher than negative: {higher_count} out of {total} ({higher_count/total:.2%})")



In [ ]:
df_all_neg_ext = pd.concat([df_soft_neg_ext, df_hard_neg_ext])
df_all_neg_ext.head()

In [12]:
# Save DataFrames to pickle format
df_soft_neg_ext.to_pickle("downloaded_model/df_soft_neg_ext.pkl")
#df_hard_neg_ext.to_pickle("downloaded_model/df_hard_neg_ext.pkl")
#df_all_neg_ext.to_pickle("downloaded_model/df_all_neg_ext.pkl")

In [4]:
# Function to load a DataFrame from pickle if the file exists
def load_df_if_exists(file_path):
    if os.path.exists(file_path):
        return pd.read_pickle(file_path)
    else:
        print(f"File not found: {file_path}")
        return None

# Load DataFrames
df_soft_neg_ext = load_df_if_exists("downloaded_model/df_soft_neg_ext.pkl")
#df_hard_neg_ext = load_df_if_exists("downloaded_model/df_hard_neg_ext.pkl")
#df_all_neg_ext = load_df_if_exists("downloaded_model/df_all_neg_ext.pkl")


In [ ]:
df_soft_neg_ext.head()

In [ ]:
#1. Create packed sequences for RNN processing
#packed_queries, packed_positives, packed_negatives = create_packed_batch(df_all_neg_ext)



In [ ]:
# # 2. Feed packed sequences to your RNN models
# query_outputs, query_hidden =your_query_rnn(packed_queries)
# pos_outputs, pos_hidden = your_document_rnn(packed_positives)
# neg_outputs, neg_hidden = your_document_rnn(packed_negatives)

In [8]:
run_hyperparameter_tuning(df_soft_neg_ext, output_dims=[100], batch_sizes=[512, 1024], gru_hidden_dims=[100,200], 
                         num_layers=[1], dropouts=[0.1], learning_rates=[1e-3], 
                         epochs=10, log_wandb=True)

Data splits: Train=47822 | Validation=15941 | Test=15941


/root/MLX_Week2/mlx_env/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(




--------------------------------------------------------------------------------
Training with: output_dim=100, batch_size=512, gru_hidden_dim=100, num_layers=1, dropout=0.1, lr=0.001
--------------------------------------------------------------------------------


Epoch 1/10 (Val): 100%|██████████| 32/32 [00:03<00:00,  8.63it/s]


Epoch 1/10, Train Loss: 0.1607, Val Loss: 0.1335, LR: 0.001000
New best model saved with validation loss: 0.1335


...

Epoch 9/10 (Val): 100%|██████████| 32/32 [00:03<00:00,  9.05it/s]


Epoch 9/10, Train Loss: 0.0392, Val Loss: 0.0651, LR: 0.001000
New best model saved with validation loss: 0.0651


Epoch 10/10 (Val): 100%|██████████| 32/32 [00:03<00:00,  8.36it/s]


Epoch 10/10, Train Loss: 0.0346, Val Loss: 0.0636, LR: 0.001000
New best model saved with validation loss: 0.0636
Final model saved at: checkpoints/final_gru_model_20250424-174424/final_gru_model_20250424-174424.pt


({'output_dim': 100,
  'batch_size': 512,
  'gru_hidden_dim': 100,
  'num_layers': 1,
  'dropout': 0.1,
  'learning_rate': 0.001,
  'val_loss': 0.07193102573929298},
 GRUTwinTowerModel(
   (query_encoder): BidirectionalGRU(
     (gru): GRU(100, 100, batch_first=True, bidirectional=True)
   )
   (doc_encoder): BidirectionalGRU(
     (gru): GRU(100, 100, batch_first=True, bidirectional=True)
   )
   (query_tower): QryTower(
     (fc1): Linear(in_features=200, out_features=128, bias=True)
     (fc2): Linear(in_features=128, out_features=64, bias=True)
     (fc3): Linear(in_features=64, out_features=100, bias=True)
     (relu): ReLU()
   )
   (doc_tower): DocTower(
     (fc1): Linear(in_features=200, out_features=128, bias=True)
     (fc2): Linear(in_features=128, out_features=64, bias=True)
     (fc3): Linear(in_features=64, out_features=100, bias=True)
     (relu): ReLU()
   )
   (dropout): Dropout(p=0.1, inplace=False)
 ))

### Twin Tower Network

In [ ]:
# model_path = os.path.join("checkpoints\final_gru_model_20250424-152045\final_gru_model_20250424-152045.pt")
# print(f"Loading model from: {model_path}")
    
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = torch.load(model_path, map_location=torch.to(device))
# print("Loaded complete model")

# model.eval()

In [14]:
# Loading the pre-trained model from WandB
api = wandb.Api()
artifact = api.artifact("nnamdi-odozi-ave-actuaries/gru-twin-tower-model/artifacts/model/final_gru_model_20250424-174424/v0")
artifact_dir = artifact.download()
model_path = f"{artifact_dir}/model.pt"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Try loading model
try:
    model = torch.load(model_path, map_location=device)
except:
    model = GRUTwinTowerModel(embedding_dim=100, gru_hidden_dim=128, output_dim=100, 
                             num_layers=1, dropout=0.1)
    model.load_state_dict(torch.load(model_path, map_location=device))

model.to(device).eval()

CommError: Invalid artifact path: nnamdi-odozi-ave-actuaries/gru-twin-tower-model/artifacts/model/final_gru_model_20250424-174424/v0

In [ ]:
# 2. Test with consecutive rows - just use a slice directly
df_slice = df_soft_neg_ext[0:3]  # Use any 3 consecutive rows

# Process dataframe slice
with torch.no_grad():
    # Move everything to device
    query_embs = torch.stack(df_slice['query_emb'].tolist()).to(device)
    query_lens = torch.tensor(df_slice['query_length'].tolist()).to(device)
    pos_embs = torch.stack(df_slice['pos_emb'].tolist()).to(device)
    pos_lens = torch.tensor(df_slice['pos_length'].tolist()).to(device)
    
    # Get encodings for all rows at once
    query_vecs, doc_vecs = model(query_embs, query_lens, pos_embs, pos_lens)
    
    # Calculate similarities
    sims = torch.nn.functional.cosine_similarity(query_vecs, doc_vecs, dim=1)
    
print("Similarities:", sims.cpu().numpy())

In [13]:
#with a random sentence:
query_test = "This is RBA"
doc_test = "This is RBA"
query_emb = text_to_embeddings(query_test, word_to_idx, embeddings, is_query=True)
doc_emb = text_to_embeddings(doc_test, word_to_idx, embeddings, is_query=False)

 # Calculate similarities
sims = torch.nn.functional.cosine_similarity(query_emb, doc_emb, dim=1)
    
print("Similarities:", sims.cpu().numpy())

TypeError: cosine_similarity(): argument 'x1' (position 1) must be Tensor, not tuple

In [ ]:
# 3. Query-only inference using just dataframe columns 
query_row = df_soft_neg_ext[0]  # Use any row
test_query_emb = query_row['query_emb'].unsqueeze(0).to(device)
test_query_len = torch.tensor([query_row['query_length']]).to(device)

# Just run through query encoder and tower
with torch.no_grad():
    query_encoded = model.query_encoder(test_query_emb, test_query_len)
    query_vector = model.query_tower(query_encoded)
    query_vector = torch.nn.functional.normalize(query_vector, p=2, dim=1) #I don't think this is needed, but let's keep it for now

print("Query vector shape:", query_vector.shape)
print("Values:", query_vector[0, :5].cpu().numpy())

In [ ]:
# Create tower instances
#qryTower = QryTower()
#docTower = DocTower()


# Define hyperparameters
batch_size = 128
num_epochs = 1 # adjust num of epochs here
dataset_size = len(df_all_neg_ext)  # or len(df_hard_neg_ext) depending on the dataset you want to use
steps_per_epoch = dataset_size // batch_size
total_steps = steps_per_epoch * num_epochs
learning_rate = 1e-3
embedding_dim = 100 #changed for glove 
margin = 0.5 

In [ ]:
# Create the dataset
dataset = TripletEmbeddingDataset(df_all_neg_ext)

In [ ]:
dataloader = DataLoader(
    dataset,
    batch_size=batch_size,
    shuffle=True,
    #num_workers=2,  # Adjust based on your machine's capabilities
    pin_memory=True  # Set to True if using GPU
)

In [ ]:
qry = torch.randn(batch_size, embedding_dim)  # Query embeddings
pos = torch.randn(batch_size, embedding_dim)  # Positive doc embeddings
neg = torch.randn(batch_size, embedding_dim)  # Negative doc embeddings

#qry = df1['q']


# Set up the AdamW optimizer
optimizer = torch.optim.AdamW([
    {'params': qryTower.parameters()},
    {'params': docTower.parameters()}
], lr=learning_rate)

# Add learning rate scheduler (ReduceLROnPlateau)
scheduler = ReduceLROnPlateau(
    optimizer,
    mode='min',       # Reduce LR when monitored value stops decreasing
    factor=0.5,       # Multiply LR by this factor when reducing
    patience=2,       # Number of epochs with no improvement after which LR will be reduced
    verbose=True      # Print message when LR is reduced
)



In [ ]:
def evaluate_model(qryTower, docTower, dataloader, device):
    qryTower.eval()
    docTower.eval()

    total = 0
    correct = 0

    for batch in dataloader:
        # Get embeddings from batch
        query_emb = batch['query']
        pos_emb = batch['positive']
        neg_emb = batch['negative']
        
        # Forward pass through towers
        query_encoded = qryTower(query_emb)
        pos_encoded = docTower(pos_emb)
        neg_encoded = docTower(neg_emb)
        
        # Calculate similarities
        pos_sim = torch.nn.functional.cosine_similarity(query_encoded, pos_encoded)
        neg_sim = torch.nn.functional.cosine_similarity(query_encoded, neg_encoded)

        correct += (pos_sim > neg_sim).sum().item()
        total += batch['query'].size(0)

    acc = correct / total
    print(f"Eval Accuracy (query closer to pos than neg): {acc:.4f}")
    return acc



total_loss = 0
    

In [ ]:
#print(f"Epoch {epoch+1}, Avg Loss: {total_loss / len(dataloader):.4f}")
evaluate_model(final_qry_tower, final_doc_tower, dataloader, device)

In [ ]:
query_emb = text_to_embeddings("What is RBA", word_to_idx, embeddings)
pos_emb = text_to_embeddings("What is RBA", word_to_idx, embeddings)

# Ensure tensors have at least two dimensions before applying mean
if query_emb.dim() == 1:
	query_emb = query_emb.unsqueeze(0)
if pos_emb.dim() == 1:
	pos_emb = pos_emb.unsqueeze(0)

query_emb = query_emb.mean(dim=0)
pos_emb = pos_emb.mean(dim=0)

print(torch.nn.functional.cosine_similarity(query_emb, pos_emb, dim=0))



In [ ]:

final_qry_tower.eval()
final_doc_tower.eval()
query_encoded = final_qry_tower(query_emb)
pos_encoded = final_doc_tower(pos_emb)
torch.nn.functional.cosine_similarity(query_encoded, pos_encoded, dim=0)